In [ ]:
import pandas as pd
import geopandas as gpd
import networkx as nx
from shapely.geometry import LineString

# 파일 경로 설정
csv_path = "./testing/광진구_clusters_route 1.csv"
geojson_path = "./testing/road.geojson"

# CSV 데이터 로드
df = pd.read_csv(csv_path)

# 도로 네트워크 로드
gdf_roads = gpd.read_file(geojson_path)

# 도로 네트워크 그래프 생성
G = nx.Graph()

# GeoJSON에서 도로 데이터 추가 (LineString을 사용하여 네트워크 구성)
for _, row in gdf_roads.iterrows():
    if isinstance(row.geometry, LineString):
        coords = list(row.geometry.coords)
        for i in range(len(coords) - 1):
            G.add_edge(coords[i], coords[i + 1], weight=row.geometry.length)

# 0번 클러스터만 필터링
df_cluster_0 = df[df['cluster'] == 0].sort_values(by='order')

# 클러스터 0의 좌표 리스트 (lat, lon 순서)
coordinates = list(zip(df_cluster_0['latitude'], df_cluster_0['longitude']))

# 도로 위의 가장 가까운 노드 찾기
nodes = [min(G.nodes, key=lambda node: (node[0] - lon)**2 + (node[1] - lat)**2) for lat, lon in coordinates]

# 최적 경로 찾기 (순서대로 경로 연결)
route_edges = []
for i in range(len(nodes) - 1):
    path = nx.shortest_path(G, source=nodes[i], target=nodes[i + 1], weight="weight")
    route_edges.extend(list(zip(path[:-1], path[1:])))

# 경로 데이터 저장
df_route = pd.DataFrame(route_edges, columns=['start_node', 'end_node'])
df_route.to_csv("cluster_0_route.csv", index=False)
